In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

In [2]:
perth = pd.read_csv('perth_clean.csv')
perth = pd.get_dummies(perth, columns=['suburb'])

train_indices, test_indices = train_test_split(perth.index, test_size=0.2, random_state=0)

perth_train = perth.loc[train_indices].copy()
perth_test = perth.loc[test_indices].copy()

In [3]:
indices = list(perth_train.index.copy())
# np.random.seed(5)
np.random.shuffle(indices)

sampled_indices = indices[:int(len(indices) * 0.2)]
sampled_perth = perth_train.loc[sampled_indices, :].copy()

print("Sampled set length", len(sampled_indices))

Sampled set length 5384


In [4]:
n = len(sampled_perth)
sampled_train_indices, sampled_valid_indices = train_test_split(np.arange(n), test_size=0.2, random_state=0)

sampled_x_train = sampled_perth.iloc[sampled_train_indices].drop('log10_price', axis=1)
sampled_y_train = sampled_perth.iloc[sampled_train_indices]['log10_price'].copy()

sampled_x_valid = sampled_perth.iloc[sampled_valid_indices].drop('log10_price', axis=1)
sampled_y_valid = sampled_perth.iloc[sampled_valid_indices]['log10_price'].copy()

In [5]:
train_data = (sampled_x_train, sampled_y_train)
valid_data = (sampled_x_valid, sampled_y_valid)

In [6]:
print("Training length of sampled data", len(sampled_x_train))
print("Validiation length of sampled data", len(sampled_x_valid))

Training length of sampled data 4307
Validiation length of sampled data 1077


$$X^i(t+1) = X^i(t) + V^i(t+1)$$

$$V^i(t+1)=wV^i(t) + c_1r_1(p^i-X^i(t))+c_2r_2(g^i-X^i(t))$$

In [7]:
class SwarmOptimizer:
    def __init__(self, n_particles, w=0.8, c1=0.1, c2=0.1):
        self.n_particles = n_particles
        self.w = w
        self.c1 = c1
        self.c2 = c2

        self.swarm = self._generate_swarm(n_particles)
        self.swarm_velocities = self._generate_swarm_velocities(n_particles)

        self.current_best_particle = None
        self.current_best_score = None

        self.global_best_particle = None
        self.global_best_score = None

    def update_swarm(self, train_data, valid_data, n_tries=1):
        for i in range(n_tries):
            print(f"Updating swarm - {i + 1} of {n_tries} updates")
            self._update_swarm(train_data, valid_data)

    def _update_swarm(self, train_data, valid_data):
        self.update_current_best_particle(train_data, valid_data)
        self.update_global_best_particle()

        self.update_particle_velocities()
        self.update_particles()

    def update_current_best_particle(self, train_data, valid_data):
        best_particle_index, best_particle_score = self.best_particle_of_swarm(train_data, valid_data)

        self.current_best_particle = self.swarm[best_particle_index].copy()
        self.current_best_score = best_particle_score

    def update_global_best_particle(self):
        if self.global_best_particle is None:
            self.global_best_particle = self.current_best_particle.copy()
            self.global_best_score = self.current_best_score

        elif self.current_best_score < self.global_best_score:
            self.global_best_particle = self.current_best_particle.copy()
            self.global_best_score = self.current_best_score

    def update_particle_velocities(self):
        w = self.w
        c1, c2 = self.c1, self.c2

        r1, r2 = np.random.rand(2)
        for particle, velocity in zip(self.swarm, self.swarm_velocities):
            for key in velocity.keys():
                p = self.current_best_particle[key]
                g = self.global_best_particle[key]
                X = particle[key]

                velocity[key] = w * velocity[key] + c1 * r1 * (p - X) + c2 * r2 * (g - X)

    def update_particles(self):
        """ Note that since the `learning_rate` and the `subsample` must be
            between 0 and 1, we must clip it ever time to make sure this is correct
            If you want to solve a different problem, then you may need to change this
        """
        for particle, velocity in zip(self.swarm, self.swarm_velocities):
            for key in velocity.keys():
                particle[key] = np.clip(particle[key] + velocity[key], 1e-5, 1)
    

    def best_particle_of_swarm(self, train_data, valid_data):
        swarm_scores = self.score_swarm(train_data, valid_data)

        best_index = np.argmin(swarm_scores)
        return best_index, swarm_scores[best_index]

    def score_swarm(self, train_data, valid_data):
        swarm_scores = []
        for i, particle in enumerate(self.swarm):
            base_model = XGBRegressor(objective='reg:squarederror', random_state=0, 
                                        n_estimators=200, max_depth=10, **particle)
            
            score = self._score_model(base_model, train_data, valid_data)
            swarm_scores.append(score)
            
            print(f"{i + 1} of {self.n_particles}: Particle {particle} has score {score}")

        return swarm_scores

    @staticmethod
    def _score_model(model, train_data, valid_data):
        x_train, y_train = train_data
        x_valid, y_valid = valid_data
        
        model.fit(x_train, y_train)
        
        y_pred = model.predict(x_valid)
        return np.sqrt(np.mean((y_valid - y_pred) ** 2))

    @staticmethod
    def _generate_swarm(n_particles):
        swarm = []
        for i in range(n_particles):
            new_particle = {}
            new_particle['learning_rate'] = np.random.uniform(low=0, high=1)
            new_particle['subsample'] = np.random.uniform(low=0, high=1)

            swarm.append(new_particle)

        return swarm

    @staticmethod
    def _generate_swarm_velocities(n_particles):
        return [{'learning_rate': 0, 'subsample': 0} for _ in range(n_particles)]

In [8]:
swarm_optimizer = SwarmOptimizer(n_particles=10, w=0.8, c1=0.1, c2=0.1)

In [9]:
swarm_optimizer.update_swarm(train_data, valid_data)

Updating swarm - 1 of 1 updates
1 of 10: Particle {'learning_rate': 0.8886824408606375, 'subsample': 0.37354936513718706} has score 0.2645394427216232
2 of 10: Particle {'learning_rate': 0.2084004527721237, 'subsample': 0.5978973760511196} has score 0.0979382481304867
3 of 10: Particle {'learning_rate': 0.8046436682852822, 'subsample': 0.5221258212643487} has score 0.14217137931937845
4 of 10: Particle {'learning_rate': 0.7523920546179751, 'subsample': 0.3794787259605792} has score 0.184693217178702
5 of 10: Particle {'learning_rate': 0.6101118551991185, 'subsample': 0.01602030104858443} has score 3863.017043048879
6 of 10: Particle {'learning_rate': 0.7095152325409344, 'subsample': 0.5519059682912614} has score 0.12756089127203957
7 of 10: Particle {'learning_rate': 0.9403559371597094, 'subsample': 0.7824331006411478} has score 0.12882397601050208
8 of 10: Particle {'learning_rate': 0.6107775097171121, 'subsample': 0.44806591933498363} has score 0.13539127988439503
9 of 10: Particle {

In [10]:
print(f"Best particle {swarm_optimizer.global_best_particle} with score {swarm_optimizer.global_best_score}")

Best particle {'learning_rate': 0.2084004527721237, 'subsample': 0.5978973760511196} with score 0.0979382481304867


In [11]:
swarm_optimizer.update_swarm(train_data, valid_data)

Updating swarm - 1 of 1 updates
1 of 10: Particle {'learning_rate': 0.8169481265673152, 'subsample': 0.3972063941454103} has score 0.2023247108633759
2 of 10: Particle {'learning_rate': 0.2084004527721237, 'subsample': 0.5978973760511196} has score 0.0979382481304867
3 of 10: Particle {'learning_rate': 0.7417710669731967, 'subsample': 0.5301157734029326} has score 0.13622889715140007
4 of 10: Particle {'learning_rate': 0.6950292767949339, 'subsample': 0.40251051625851575} has score 0.15085627031197804
5 of 10: Particle {'learning_rate': 0.5677522271309416, 'subsample': 0.07737802285979305} has score 37.253570669083885
6 of 10: Particle {'learning_rate': 0.6566737260210121, 'subsample': 0.5567556661471581} has score 0.12772468879700685
7 of 10: Particle {'learning_rate': 0.8631727606459307, 'subsample': 0.7629741940437785} has score 0.13045814642965103
8 of 10: Particle {'learning_rate': 0.5683476897708015, 'subsample': 0.4638653333945885} has score 0.12708456400751378
9 of 10: Particle

In [12]:
print(f"Best particle {swarm_optimizer.global_best_particle} with score {swarm_optimizer.global_best_score}")

Best particle {'learning_rate': 0.04306688755705752, 'subsample': 0.45645298610669627} with score 0.08970690579896985


In [13]:
swarm_optimizer.update_swarm(train_data, valid_data, n_tries=3)

Updating swarm - 1 of 3 updates
1 of 10: Particle {'learning_rate': 0.688810605230655, 'subsample': 0.4215484821936451} has score 0.15507472609018771
2 of 10: Particle {'learning_rate': 0.19328526363654866, 'subsample': 0.5849661923353694} has score 0.0965641572669653
3 of 10: Particle {'learning_rate': 0.6275957822356149, 'subsample': 0.5297733072873151} has score 0.12746150169789633
4 of 10: Particle {'learning_rate': 0.5895350968658785, 'subsample': 0.4258674978355289} has score 0.13293865479295106
5 of 10: Particle {'learning_rate': 0.4858965386597291, 'subsample': 0.1611201381915775} has score 0.3149685246177858
6 of 10: Particle {'learning_rate': 0.5583031191298593, 'subsample': 0.5514655140289605} has score 0.11585164436459759
7 of 10: Particle {'learning_rate': 0.7264501831605468, 'subsample': 0.719384168358412} has score 0.12219088462865718
8 of 10: Particle {'learning_rate': 0.4863814091813481, 'subsample': 0.4758272101620747} has score 0.11876840442325756
9 of 10: Particle {

In [14]:
swarm_optimizer.update_swarm(train_data, valid_data)

Updating swarm - 1 of 1 updates
1 of 10: Particle {'learning_rate': 0.16559099972230232, 'subsample': 0.4879387656503457} has score 0.09594760104547065
2 of 10: Particle {'learning_rate': 0.10372060796878124, 'subsample': 0.5083428033397314} has score 0.09062555417305108
3 of 10: Particle {'learning_rate': 0.15794782840105054, 'subsample': 0.501451520224029} has score 0.09224949717546857
4 of 10: Particle {'learning_rate': 0.15319564053784734, 'subsample': 0.4884780300780606} has score 0.09296904253674342
5 of 10: Particle {'learning_rate': 0.1402555188957198, 'subsample': 0.45542215688258075} has score 0.09565136147617796
6 of 10: Particle {'learning_rate': 0.14929607267407585, 'subsample': 0.5041599696542566} has score 0.09429065853538843
7 of 10: Particle {'learning_rate': 0.1702906088774425, 'subsample': 0.5251259870382824} has score 0.09343593194080406
8 of 10: Particle {'learning_rate': 0.14031605894605464, 'subsample': 0.49471590877785176} has score 0.09410553978966164
9 of 10: 

In [15]:
print(f"Best particle {swarm_optimizer.global_best_particle} with score {swarm_optimizer.global_best_score}")

Best particle {'learning_rate': 0.08691132105446604, 'subsample': 0.4939623033492828} with score 0.08892643429148894


In [16]:
%%time
best_parameters = swarm_optimizer.global_best_particle

final_model = XGBRegressor(objective='reg:squarederror', random_state=0, 
                           n_estimators=200, max_depth=10, **best_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.007392323683920141
CPU times: user 2min 39s, sys: 254 ms, total: 2min 39s
Wall time: 2min 38s


In [ ]:
%%time
# Best parameters by genetic algorithm - 5 generations each with 20 population size (100 total)

best_parameters = {'learning_rate': 0.06000000000000001, 'max_depth': 11, 'subsample': 0.7000000000000001}

final_model = XGBRegressor(objective='reg:squarederror', random_state=0, n_estimators=200, **best_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.00724586724861683
CPU times: user 2min 33s, sys: 133 ms, total: 2min 33s
Wall time: 2min 33s


In [ ]:
%%time
# Best parameters by coordinate descent - 153 different models sampled

best_parameters = {'learning_rate': 0.09, 'max_depth': 11, 'n_estimators': 170, 'subsample': 0.8}

final_model = XGBRegressor(objective='reg:squarederror', random_state=0, **best_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.007313312056798277
CPU times: user 2min, sys: 96.9 ms, total: 2min
Wall time: 2min
